# READING POINT CLOUD REFERENCE DATA FOR POSE MAPPING
In this notebook, we evaluate the RDF GRAPHS in a session repository.
For every POINTCLOUDNODE resource, a POINTCLOUDNODE metadata class is created and its data is loaded. 
The following files will be imported and represented in memory
>nodelist=PointCloudNode[]

>This codebase operates on the scan2bim.yml environment (python 3.8)

In [57]:
# import APIs
import rdflib
from rdflib import Graph, plugin
from rdflib.serializer import Serializer #pip install rdflib-jsonld https://pypi.org/project/rdflib-jsonld/
from rdflib import Graph
from rdflib import URIRef, BNode, Literal
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD

import os.path, time
import importlib
import PIL
import PIL.Image as PILimage
from PIL import ImageDraw, ImageFont, ImageEnhance
from PIL.ExifTags import TAGS, GPSTAGS
import xml.etree.ElementTree as ET
import numpy as np
import open3d as o3d

#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

## 1. INITIALIZE SESSION

In [58]:
## INPUTS
projectPath= os.path.abspath(os.path.join(os.getcwd(), os.pardir))+"\\tests"#"D:\\Data\\2018-06 Werfopvolging Academiestraat Gent" 
sessionPath = projectPath + "\\Samples3" #"K:\Projects\2025-03 Project FWO SB Jelle\7.Data\21-11 House Maarten\RAW data\session_22-03-13 canon

## 2. READ AND MERGE All THE RDF GRAPHS IN THE SESSION (save for feature.ttl files)

In [60]:
## read all graph files in a session and merge them
allSessionFilePaths=ut.get_list_of_files(sessionPath) 
rdfGraphPaths=[]

# collect all RDF graphs in the session
sessionGraph = Graph()
for path in allSessionFilePaths:        
    if 'pcdGraph' in path: 
        rdfGraphPaths.append(path)
        sessionGraph.parse(path)     
        print('Graph found with '+str(len(sessionGraph)) +' triples')

Graph found with 67 triples


In [61]:
#test
print(sessionGraph.serialize())

@prefix e57: <http://libe57.org#> .
@prefix gom: <https://w3id.org/gom#> .
@prefix openlabel: <https://www.asam.net/index.php?eID=dumpFile&t=f&f=3876&token=413e8c85031ae64cc35cf42d0768627514868b2f#> .
@prefix v4d: <https://w3id.org/v4d/core#> .

<http://academiestraat_week_22_19> a "<class 'geomapi.pointcloudnode.PointCloudNode'>" ;
    e57:cartesianBounds """[-2.83604126e+01  5.53711433e+01  5.41933060e+01  1.07839798e+02
 -1.84059143e-03  1.11153946e+01]""" ;
    e57:cartesianTransform """[[ 1.          0.          0.          4.48137623]
 [ 0.          1.          0.         80.6841877 ]
 [ 0.          0.          1.          5.13496894]
 [ 0.          0.          0.          1.        ]]""" ;
    gom:coordinateSystem "local" ;
    v4d:PointCount "11934055" ;
    v4d:accuracy "0.002" ;
    v4d:guid "f584d5d1-d05a-11ec-a6fa-c8f75043ce59" ;
    v4d:linkedNodes "['http://academiestraat_week_22_20', 'http://academiestraat_week_22_21', 'http://academiestraat_week_22_22']" ;
    v4d:orien

# 3. A PointCloudNode is created per point cloud subject in the joint RDF Graph

In [62]:
nodelist=tl.graph_to_nodes(sessionGraph,sessionPath=sessionPath)
print (str(len(nodelist))+' Nodes succesfully created from the session!')

6 Nodes succesfully created from the session!


In [67]:
test=nodelist[0].linkedNodes
print(test)
myNodeList=tl.get_linked_nodes(nodelist[0],sessionGraph,getGeometry=False, sessionPath=sessionPath)
subjects=[node.subject for node in myNodeList]
print(subjects)

[rdflib.term.URIRef('http://academiestraat_week_22_20'), rdflib.term.URIRef('http://academiestraat_week_22_21'), rdflib.term.URIRef('http://academiestraat_week_22_22')]
[rdflib.term.URIRef('http://academiestraat_week_22_20'), rdflib.term.URIRef('http://academiestraat_week_22_21'), rdflib.term.URIRef('http://academiestraat_week_22_22')]


## 4. IMPORT O3D.POINTCLOUDS
import o3d.geometry.pointclouds for each pointcloudnode if present

In [64]:
counter=0
for node in nodelist:
    if node.get_geometry():
        print(str(node.name)+' succusfully imported')
        counter +=1

print (str(counter)+' of '+str(len(nodelist))+ ' PointCloudNodes have their o3d.geometry.pointclouds imported!')

academiestraat_week_22_19 succusfully imported
academiestraat_week_22_20 succusfully imported
academiestraat_week_22_21 succusfully imported
academiestraat_week_22_22 succusfully imported
academiestraat_week_22_23 succusfully imported
academiestraat_week_22_24 succusfully imported
6 of 6 PointCloudNodes have their o3d.geometry.pointclouds imported!


In [65]:
o3d.visualization.draw_geometries([nodelist[1].pcd])